In [60]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [61]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [62]:
params = {"grupo":"9"}

In [63]:
r = requests.get(base_url, params=params)

In [64]:
page = BeautifulSoup(r.text, 'html.parser')

In [65]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [66]:
__EVENTVALIDATION

'/wEdAAw2UaVvvzuVRSbbkO3+L8yy4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc2mSK+Opr3Aos6JmW2p54fU6aD8Bg=='

In [67]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [68]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUBOR4IaWRmR3J1cG8CCR4IYnVzY2FQb3JkFgJmD2QWAgIDD2QWAgIBDxYCHgZhY3Rpb24FHi9kYWRvc2FiZXJ0b3MvY29uc3VsdGE/Z3J1cG89ORYOAgIPEA8WBh4NRGF0YVRleHRGaWVsZAUKTm9tZVBhZ2luYR4ORGF0YVZhbHVlRmllbGQFCVVybFBhZ2luYR4LXyFEYXRhQm91bmRnFgQeCW9ua2V5ZG93bgVDaWYoSXNFbnRlcktleShldmVudCkpe1JlZGlyZWNpb25hckFjZXNzb1JhcGlkbyh0aGlzKTtyZXR1cm4gZmFsc2U7fR4Hb25jbGljawUfUmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpOxAVBg5BY2Vzc28gUsOhcGlkbxZTZWNyZXRhcmlhcyBlIMOTcmfDo29zEVNlcnZpw6dvcyBPbi1MaW5lGFNlcnZpw6dvcyBwYXJhIGEgRW1wcmVzYRlTZXJ2acOnb3MgcGFyYSBvIENpZGFkw6NvE1NpdGVzIGRhIFByZWZlaXR1cmEVBgANL3NlY3JldGFyaWFzLxkvbGlzdGEtc2Vydmljb3Mtb24tbGluZS8yES9zZXJ2aWNvcy9lbXByZXNhES9zZXJ2aWNvcy9jaWRhZGFvHC9saXN0YS1zaXRlcy1kYS1wcmVmZWl0dXJhLzUUKwMGZ2dnZ2dnZGQCBA8PZBYCHwcFSXJldHVybiBWYWxpZGFyUGVzcXVpc2EoJ3R4dFBlc3F1aXNhVG9wbycsJ2N0bDAwJGxua1Blc3F1aXNhVG9wbycsIGV2ZW50KTtkAgYPDxYCHgRUZXh0BQ5Tw6FiLCAxMy8xMC8xOGRkAgcPDxYCHghJbWFnZVVybAUVfi9pbWFnZW0vdGVtcG8vcGMuZ2lmZGQCCA8PFgIfCQUEMjXCumRkAgkPDxYCHwkFBDE

In [69]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [70]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [71]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [72]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [73]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [74]:
print(md5(a_tags[0].text))

3c76ee433184c23533b1e1a6e03ccb62


In [75]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [76]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [77]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 5/5 [00:00<00:00, 8518.08it/s]


In [78]:
databases

[{'database_id': '3c76ee433184c23533b1e1a6e03ccb62',
  'database_title': 'FILA - Base De Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/PretendentesCohab/2018-03-12_FILA_-_Base De Dados.csv',
  'database_date': datetime.date(2018, 3, 12)}]

In [79]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/PretendentesCohab/2018-03-12_FILA_-_Base De Dados.csv', encoding="ISO-8859-1", delimiter=';', error_bad_lines=False)

TimeoutError: [Errno 60] Operation timed out

In [ ]:
test.fillna('').head()